In [114]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn



In [115]:
# train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp32.csv' ,delimiter=',')
# test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp32.csv' ,delimiter=',')

train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')





In [116]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [117]:
# Separate features and target
X = train.drop(columns=['price'])
y = ((train['price'])) # talk about this 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [118]:
# # Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [119]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# # Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [150]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [151]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

def rmsle_loss(y_pred, y_true):
    # Ensure the inputs are float tensors (required for MSELoss and logarithmic operations)

    
    # Compute the RMSLE using the logarithmic differences
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))
    # return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

In [144]:
from torch.optim.lr_scheduler import CyclicLR

def train_model(model, train_loader, test_loader, num_epochs=5):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-6, momentum=0.8)
    scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=0.1, step_size_up=5*len(train_loader),
                     mode='triangular', cycle_momentum=False)


    min_train_rmsle = float('inf')
    min_test_rmsle = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = rmsle_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate at each batch

            total_train_loss += loss.item()

        avg_train_rmsle = total_train_loss / len(train_loader)

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss = rmsle_loss(outputs, targets)
                total_test_loss += test_loss.item()

        avg_test_rmsle = total_test_loss / len(test_loader)

        if avg_train_rmsle < min_train_rmsle:
            min_train_rmsle = avg_train_rmsle
        if avg_test_rmsle < min_test_rmsle:
            min_test_rmsle = avg_test_rmsle

        print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , Current LR = {scheduler.get_last_lr()}')

    print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
    print(f'Minimum Test RMSLE so far: {min_test_rmsle}')


In [138]:
# def train_model(model, train_loader, test_loader, num_epochs=5):
#     model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
#     scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, min_lr=1e-6)

#     # Initialize minimum loss to a large value
#     min_train_rmsle = float('inf')
#     min_test_rmsle = float('inf')

#     for epoch in range(num_epochs):
#         model.train()
#         total_train_loss = 0
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.to(device), targets.to(device)

#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = rmsle_loss(outputs, targets)
#             loss.backward()
#             optimizer.step()

#             total_train_loss += loss.item()

#         avg_train_rmsle = total_train_loss / len(train_loader)

#         model.eval()
#         total_test_loss = 0
#         with torch.no_grad():
#             for inputs, targets in test_loader:
#                 inputs, targets = inputs.to(device), targets.to(device)
#                 outputs = model(inputs)
#                 test_loss = rmsle_loss(outputs, targets)
#                 total_test_loss += test_loss.item()

#         avg_test_rmsle = total_test_loss / len(test_loader)
        
#         # Update the learning rate scheduler
#         scheduler.step(avg_test_rmsle)

#         if avg_train_rmsle < min_train_rmsle:
#             min_train_rmsle = avg_train_rmsle
#         if avg_test_rmsle < min_test_rmsle:
#             min_test_rmsle = avg_test_rmsle

#         print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , {scheduler.get_last_lr()}')
    
#     print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
#     print(f'Minimum Test RMSLE so far: {min_test_rmsle}')

In [124]:
num_features = X_train.shape[1]


In [125]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedMLP(nn.Module):
    def __init__(self, num_features):
        super(AdvancedMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.1)
        
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.final_layer = nn.Linear(128, 1)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = F.leaky_relu(self.bn4(self.fc4(x)))
        x = self.final_layer(x)
        return x


In [126]:
model = AdvancedMLP(num_features).to(device)
train_model(model, train_loader, test_loader, num_epochs=10)
# Minimum Train RMSLE so far: 0.6773702232499507
# Minimum Test RMSLE so far: 0.678769255297098

Epoch 1: Train RMSLE = 0.7514917956175907, Test RMSLE = 0.691889699742976 , Current LR = [0.020080000000000018]
Epoch 2: Train RMSLE = 0.6929407155532386, Test RMSLE = 0.6899763355093947 , Current LR = [0.04006]
Epoch 3: Train RMSLE = 0.6914681243433587, Test RMSLE = 0.6898108521099869 , Current LR = [0.06004000000000001]
Epoch 4: Train RMSLE = 0.6908821830577122, Test RMSLE = 0.6894941653712208 , Current LR = [0.08002]
Epoch 5: Train RMSLE = 0.6907152118919099, Test RMSLE = 0.6900911300057867 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6903489363416979, Test RMSLE = 0.6894711216729648 , Current LR = [0.08002]
Epoch 7: Train RMSLE = 0.689899885309937, Test RMSLE = 0.6891402827807699 , Current LR = [0.06004000000000001]
Epoch 8: Train RMSLE = 0.6895308730073392, Test RMSLE = 0.6893058754753077 , Current LR = [0.04006]
Epoch 9: Train RMSLE = 0.6890082106383297, Test RMSLE = 0.6892872435780095 , Current LR = [0.020080000000000018]
Epoch 10: Train RMSLE = 0.688635042431471, Test RMSLE = 

In [160]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomMLP(nn.Module):
    def __init__(self, num_features):
        super(CustomMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.2)
        
        self.final_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        
        x = self.dropout(x)
        x = self.final_layer(x)
        return x


In [ ]:
model10 = CustomMLP(num_features).to(device)
train_model(model10, train_loader, test_loader, num_epochs=10)

# Minimum Train RMSLE so far: 0.6754996292528404
# Minimum Test RMSLE so far: 0.6798468978210392

In [167]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicMLP(nn.Module):
    def __init__(self, num_features, max_first_layer_size, min_layer_size=32, output_size=1):
        super(DynamicMLP, self).__init__()
        layers = []
        current_size = max_first_layer_size
        # Ensure the loop creates layers correctly by adjusting input and output sizes
        input_size = num_features  # Starting number of features
        while current_size > min_layer_size:
            next_size = max(current_size // 2, min_layer_size)
            layers.append(nn.Linear(input_size, current_size))
            layers.append(nn.BatchNorm1d(current_size))
            # layers.append(nn.Dropout(0.05))  # Using a constant dropout rate
            layers.append(nn.ReLU())
            input_size = current_size  # Update input size for the next layer
            current_size = next_size
        
        # Append the final layer
        layers.append(nn.Linear(input_size, output_size))

        # Register all layers
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# Example of using the DynamicMLP class


In [168]:
max_first_layer_size = 1024  # Adjusted for demonstration
model = DynamicMLP(num_features, max_first_layer_size).to(device)
print(model)


DynamicMLP(
  (layers): Sequential(
    (0): Linear(in_features=48, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=128, bias=True)
    (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=128, out_features=64, bias=True)
    (13): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [169]:
train_model(model, train_loader, test_loader, num_epochs=10)
# Minimum Train RMSLE so far: 0.6845218561584729
# Minimum Test RMSLE so far: 0.6898618382582543

Epoch 1: Train RMSLE = 0.7755220616465888, Test RMSLE = 0.6898555177126824 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6901063022337403, Test RMSLE = 0.6879812699859111 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.6891731141734524, Test RMSLE = 0.6884494067505054 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6888162409863118, Test RMSLE = 0.6885505358649272 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6884706852808715, Test RMSLE = 0.6893075191993177 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6880786281454072, Test RMSLE = 0.6883744412637943 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6871726254996301, Test RMSLE = 0.6887541645783841 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6861238504724139, Test RMSLE = 0.6883898411504795 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6847569100117662, Test RMSLE = 0.6892120573867908 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6827360125809078, Test

In [170]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer

# Assume `train` is your DataFrame loaded with the data
X = train.drop(columns=['price'])
y = np.log1p(train['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [171]:
def rmsle(preds, dtrain):
    labels = dtrain.get_label()
    # prevent negative predictions
    preds = np.clip(preds, a_min=0, a_max=None)
    return 'RMSLE', np.sqrt(np.mean(np.power((preds-labels), 2)))


In [ ]:
from scipy.stats import randint, uniform

param_dist = {
    'max_depth': randint(0, 100),
    'min_child_weight': randint(0, 100),
    'subsample': uniform(0.6, 0.6),
    'colsample_bytree': uniform(0.6, 0.6),
    'learning_rate': uniform(0.01, 0.8),
    'n_estimators': randint(0,1000)
}

# Create the RMSLE scorer
def rmsle_scorer(y_true, y_pred):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

rmsle_scoring = make_scorer(rmsle_scorer, greater_is_better=False)

# Initialize the XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=10)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, 
                                   n_iter=100, scoring=rmsle_scoring, cv=3, verbose=3, random_state=42,n_jobs=10)

# Fit the model
random_search.fit(X_train, y_train)


In [ ]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
test_rmsle = rmsle_scorer(y_test, y_pred)

print("Best model parameters:", random_search.best_params_)
print("Test RMSLE:", test_rmsle)
